Если хотите работать на Google Colaboratory

Для получения ключа доступа для API переходите
*   https://www.kaggle.com/lkatran (где "lkatran" - ваш user name)
*   нажимаем Edit Profile
*   находим ниже раздел API
*   нажимаем Create New API Token
*   загружается файл kaggle.json
*   в нем ваш ключ

Импорт библиотек

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import EfficientNetB7
## ______________ КОНЕЦ БЛОКА С ИМПОРТАМИ АРХИТЕКТУР ____________________

# импорт других полезных инструментов: слоев, оптимизаторов, функций обратной связи
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

## Настраиваем TPU конфигурацию

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Проверяем существующее оборудование и выбираем соответствующую стратегию использования вычислений
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # онаружение TPU. Параметры не требуются если установленна переменная окружения TPU_NAME. На Kaggle это всегда True.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # если TPU отсутствует, то испльзуем стратегию по умолчанию для TF (CPU or GPU)

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Путь к данным. Если работаете на Google Colaboratory, то замените KaggleDatasets().get_gcs_path() на путь к данным, который будет у вас
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Конфигурация
EPOCHS = 40
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMG_SIZE = (512, 768)

Посмотрим на данные

## Загружаем метки классов и пути к изображениям

In [ ]:
# функция, которая превращает айди картинки в полный путь к ней
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

train_labels = train.loc[:, 'healthy':].values

## если планируете обучать модель с валидирующим набором данных
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size=0.15, random_state=2020)

In [ ]:
from matplotlib import pyplot as plt

# создаем сетку 2 на 5, для более компактного отображения символов и задаем размер их отображения
f, ax = plt.subplots(3, 6, figsize=(18, 7))
ax = ax.flatten()
# отрисовываем в цикле найденные топ N изображений частей графем
for i in range(18):
    img = plt.imread(f'../input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg')
    ax[i].set_title(train[train['image_id']==f'Train_{i}'].melt()[train[train['image_id']==f'Train_{i}'].melt().value == 1]['variable'].values[0])
    ax[i].imshow(img)
print(img.shape)

## Создаем объекты наборов данных

A `tf.data.Dataset` объект нужен для того, чтобы модель бесперебойно работала на TPUs

In [ ]:
import tensorflow as tf

# функция, которая читает изображение из файла и преобразовывает его к нужному размеру, а так же нормализует
def decode_image(filename, label=None):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, np.uint8)
    image = tf.image.resize_with_pad(image, IMG_SIZE[0], IMG_SIZE[1])
    
    if label is None:
        return image
    else:
        return image, label
# функция расширения данных


def data_augment(image, label=None):
#     image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    
    return image, label

Создаем объекты наборов данных для обучения, валидации и теста

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))


train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO) # если напишите свою функцию расширения данных data_augment 
    .repeat()
    .shuffle(1024)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
# если планируете обучать модель с валидирующим набором данных
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

## Строим и обучаем модель

### Вспомогательные функции

In [ ]:
LR_START = 0.0001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.0001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 4
LR_EXP_DECAY = .8

STEPS_PER_EPOCH = len(train_paths) // BATCH_SIZE * 2
VAL_STEPS_PER_EPOCH = len(valid_paths) // BATCH_SIZE

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

### Загружаем модель на TPU

In [ ]:
!pip install efficientnet

In [ ]:
from tensorflow.keras.layers import LeakyReLU, GaussianDropout
from tensorflow.keras.models import Sequential
from efficientnet.tfkeras import EfficientNetB7

def get_model(use_model):
    model = Sequential()

    base_model = use_model(weights='noisy-student', 
                      include_top=False, pooling='avg',
                      input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
#     for layer in base_model.layers:
#         if any(name in layer.name for name in ['block1', 'block2', 'block3', 'block4']):
#             layer.trainable = False
            
    model.add(base_model)
    model.add(Dense(4, activation='softmax'))
    
    return model


with strategy.scope():    
    model = get_model(EfficientNetB7) # тут подставить свою модель
        
model.compile(
    optimizer='nadam',
    loss = 'categorical_crossentropy',
    metrics=['categorical_accuracy']
)
# Визуализируем архитектуру модели
# tf.keras.utils.plot_model(
#     model, to_file='model.png', show_shapes=True, show_layer_names=True,
# )

In [ ]:
model.summary()

### Запускаем процесс обучения

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

history = model.fit(train_dataset, validation_data=valid_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, validation_steps=VAL_STEPS_PER_EPOCH,
          epochs=EPOCHS, 
          callbacks=[lr_callback, EarlyStopping(patience=10, restore_best_weights=True, min_delta=1e-4)],
          workers=3)

## Проверка качества модели

In [ ]:
plt.plot(history.history['categorical_accuracy'], 
         label='Оценка точности на обучающем наборе')
plt.plot(history.history['val_categorical_accuracy'], 
         label='Оценка точности на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Оценка точности')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], 
         label='Оценка потерь на обучающем наборе')
plt.plot(history.history['val_loss'], 
         label='Оценка потерь на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Оценка потерь')
plt.legend()
plt.show()

## Делаем прогноз тестовых данных и готовим представление данных для проверки

Когда вы получите несколько моделей с высокими баллами, можно поробовать их объеденить

probs = (model1.predict(test_dataset)+model.predict(test_dataset))/2

где можно использовать и больше моделей, но тогда делить не на 2, а на количество использованных моделей

In [ ]:
probs = model.predict(test_dataset)
# probs = (probs.max(axis=1,keepdims=1) == probs).astype(int)

# assert probs.sum() == probs.shape[0]

sub.loc[:, 'healthy':] = probs
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
# создаем сетку 2 на 5, для более компактного отображения символов и задаем размер их отображения
f, ax = plt.subplots(nrows=3, ncols=5, figsize=(18, 8))
# отрисовываем в цикле найденные топ N изображений частей графем
for i in range(15):
    img = plt.imread(f'../input/plant-pathology-2020-fgvc7/images/Test_{i}.jpg')
    ax[i // 5, i % 5].set_title(sub[sub['image_id']==f'Test_{i}'].melt().iloc[1:][sub[sub['image_id']==f'Test_{i}'].melt().iloc[1:].value >= 0.8]['variable'].values[0])
    ax[i // 5, i % 5].imshow(img)
print(img.shape)